# Clasificación multiclase
##### Nombre: Soria Colque Franz Ronald
 
github: https://github.com/FranzDS01/InteligenciaArtifical/blob/main/Examen%20Laboratorio/Clasificacion_multiclase.ipynb

 Alumno: Soria Colque Franz Ronald
 Carrera: Ingenieria de Sistemas
 Grupo laboratorio: Miercoles
 Materia: sis420

Dataset :https://www.kaggle.com/datasets/uciml/glass

Calsificacion de tipo de vidrio 
Columnas:
RI: índice de refracción
Na: Sodio (unidad de medida: porcentaje en peso en óxido correspondiente, como son los atributos 4-10)
Mg: Magnesio
Al: Aluminio
Si: Silicio
K: Potasio
Ca: Calcio
Ba: Bario
Fe: Hierro
Tipo de vidrio:6 tipos

Mejor valor de lamnda=0.002
Mejor valor de maxIter=1500
Mejor valor de theta :
[[-1.35484651e-02 -2.43040551e-02 -1.81421535e-01 -3.68109088e-02
  -1.90819293e-02 -9.84737673e-01 -7.92232951e-03 -1.22120174e-01
  -2.88473920e-03 -3.30582550e-04]
 [ 1.89656186e+01  1.93975898e-01 -1.12016860e-01  1.86676605e-01
  -3.50816074e-01 -2.42674154e-01 -1.25760901e-01 -8.02184625e-02
  -4.52411897e-01 -1.09966833e+00]
 [-1.61651377e+00 -4.27264539e-02 -3.63512451e-01  9.03462918e-02
   3.25894494e-02  6.16630311e-02  3.28827025e-03  1.09738519e-01
   9.81226155e-02  2.75455887e+00]
 [ 2.24467555e+01 -6.63300752e-01 -2.02616906e-01  1.19692581e-01
  -2.97595063e-01 -2.87098144e-01  1.94180693e-02 -2.74849196e-02
   1.08209975e-01 -9.01805344e-01]
 [-1.35484651e-02 -2.43040551e-02 -1.81421535e-01 -3.68109088e-02
  -1.90819293e-02 -9.84737673e-01 -7.92232951e-03 -1.22120174e-01
  -2.88473920e-03 -3.30582550e-04]
 [-2.10186903e+01  9.10684499e-01  1.75128729e-01 -1.38587569e-01
   7.06298698e-01  1.69360145e-01  3.97495922e-01  1.41021682e-01
  -6.12505514e-01  1.23209096e+00]]

  Con datos originales sin los sinteticos el nivel precision alcanza un maximo de 54.14%





In [59]:

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot
from scipy import optimize
from scipy.io import loadmat
%matplotlib inline

Verfficacion de cantidad de clasificaciones que tiene la columna y ="type"

In [60]:
data =pd.read_csv('./Train.csv')
value = list(data["Segmentation"].unique())
print("numero de clases",value)
data.info()

numero de clases ['D', 'A', 'B', 'C']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [61]:
def csvAdapter(dataset,modificarColumnas,eliminarColumnas):
    data =pd.read_csv(dataset,encoding='latin-1')
    columna = data.columns
    data = data.dropna()
    for index in modificarColumnas:
        name = columna[index]
        value = list(data[name].unique())
        data[name] = data[name].map(dict(zip(value,[i for i in range(len(value))])))
    #print(data.head())
    data = data.drop([columna[index]for index in eliminarColumnas], axis=1)
    return data
rutadatset='./Train.csv'
columnaCategoricas = [i for i in range(0, 11) if not i in [3,6,8]]  # poner columnas que no son categoricas
print(columnaCategoricas)
eliminarColumnas=[0,9]#columna a eliminar
data = csvAdapter(rutadatset, columnaCategoricas,eliminarColumnas)

[0, 1, 2, 4, 5, 7, 9, 10]


In [62]:
value = list(data["Segmentation"].unique())
print("numero de clases",value)

data.info()

numero de clases [0, 1, 2, 3]
<class 'pandas.core.frame.DataFrame'>
Index: 6665 entries, 0 to 8067
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           6665 non-null   int64  
 1   Ever_Married     6665 non-null   int64  
 2   Age              6665 non-null   int64  
 3   Graduated        6665 non-null   int64  
 4   Profession       6665 non-null   int64  
 5   Work_Experience  6665 non-null   float64
 6   Spending_Score   6665 non-null   int64  
 7   Family_Size      6665 non-null   float64
 8   Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(7)
memory usage: 520.7 KB


In [63]:
# Crear tres DataFrames diferentes según los valores de la columna 4
#data_0 = data[data['Type'] == 0]
#data_1 = data[data['Type'] == 1]
#data_2 = data[data['Type'] == 2]
#data_3 = data[data['Type'] == 3]
#data_4 = data[data['Type'] == 4]
#data_5 = data[data['Type'] == 5]


In [64]:
#columna_i =data_0.iloc[:,5]
#muestras_i=np.random.choice(columna_i,1,replace=True)
#print(muestras_i)

In [65]:
import random
def DatosSinteticosNew(data,listaData,n_filas,clases):
    nuevos_datos=np.empty([n_filas,len(data.columns)])
    ColmunaY = np.random.randint(0,clases, n_filas)
    #print(ColmunaY)
    for i in range(len(data.columns)-1):
        for j in range(0,n_filas):
            pos=ColmunaY[j]
            columna_i =listaData[pos].iloc[:,i]
            muestras_i=np.random.choice(columna_i,1,replace=True)

            probalidad_de_cambio = random.random()
            if probalidad_de_cambio<0.2:
                muestras_i=muestras_i+probalidad_de_cambio
            elif probalidad_de_cambio<0.1:
                muestras_i=muestras_i-probalidad_de_cambio
            if(i == 0):
                muestras_i=muestras_i.round(5)
            else:
                muestras_i=muestras_i.round(2)
            #np.random.seed()
            nuevos_datos[j:,i]=muestras_i
            nuevos_datos[j:,9]=ColmunaY[j]
    columnas =list(data.columns)
    print(nuevos_datos)
    nuevos_datos_df=pd.DataFrame(nuevos_datos,columns=columnas)
    dataExtend =pd.concat([data,nuevos_datos_df],axis=0)
    #print(dataExtend.tail())
    return dataExtend


In [66]:
ListaData=[]
for i in range(0,len(value)):
    ListaData.append(data[data['Segmentation'] == i])

In [67]:
ListaData[0]

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Segmentation
0,0,0,22,0,0,1.0,0,4.0,0
7,1,0,33,1,0,1.0,0,3.0,0
8,1,1,61,1,1,0.0,0,3.0,0
11,0,0,19,0,0,4.0,0,4.0,0
16,1,0,32,0,5,9.0,0,5.0,0
...,...,...,...,...,...,...,...,...,...
8055,0,0,18,0,0,0.0,0,2.0,0
8057,0,1,85,0,2,1.0,0,1.0,0
8059,0,0,39,1,0,3.0,0,2.0,0
8064,0,0,35,0,8,3.0,0,4.0,0


In [68]:
data=DatosSinteticosNew(data,ListaData,10000,4)
data.to_csv('Prueba.csv',index=False)

IndexError: index 9 is out of bounds for axis 1 with size 9

In [ ]:
def DatosSinteticosNew(data,listaData,n_filas,clases):
    nuevos_datos=np.empty([n_filas,len(data.columns)])
    ColmunaY = np.random.randint(0,clases, n_filas)
    print(ColmunaY)
    for i in range(len(data.columns)-1):
        for j in range(0,n_filas):
            pos=ColmunaY[j]
            columna_i =listaData[pos].iloc[:,i]
            muestras_i=np.random.choice(columna_i,1,replace=True)
            if(pos == 0):
                muestras_i=muestras_i.round(5)
            else:
                muestras_i=muestras_i.round(3)
            #np.random.seed()
            nuevos_datos[j:,i]=muestras_i
            nuevos_datos[j:,9]=ColmunaY[j]
    columnas =list(data.columns)
    print(nuevos_datos)
    nuevos_datos_df=pd.DataFrame(nuevos_datos,columns=columnas)
    dataExtend =pd.concat([data,nuevos_datos_df],axis=0)
    #print(dataExtend.tail())
    return dataExtend



0.11063179320448269


In [ ]:
"""
def DatosSinteticosNew(data,data_0,data_1, data_2, data_3, data_4, data_5):
    n_filas=10000
    nuevos_datos=np.empty([n_filas,len(data.columns)])
    ColmunaY = np.random.randint(0, 6, n_filas)
    print(ColmunaY)
    for i in range(len(data.columns)-1):
        for j in range(0,n_filas):
            if ColmunaY[j]==0:
                columna_i =data_0.iloc[:,i]

            if ColmunaY[j]==1:
                columna_i =data_1.iloc[:,i]
                
            if ColmunaY[j]==2:
                columna_i =data_2.iloc[:,i]
                
            if ColmunaY[j]==3:
                columna_i =data_3.iloc[:,i]
               
            if ColmunaY[j]==4:
                columna_i =data_4.iloc[:,i]

            if ColmunaY[j]==5:
                columna_i =data_5.iloc[:,i]

            muestras_i=np.random.choice(columna_i,1,replace=True)
            if(ColmunaY[j] == 0):
                muestras_i=muestras_i.round(5)
            else:
                muestras_i=muestras_i.round(3)
            #np.random.seed()
            nuevos_datos[j:,i]=muestras_i
            nuevos_datos[j:,9]=ColmunaY[j]
    columnas =list(data.columns)
    print(nuevos_datos)
    nuevos_datos_df=pd.DataFrame(nuevos_datos,columns=columnas)
    dataExtend =pd.concat([data,nuevos_datos_df],axis=0)
    #print(dataExtend.tail())
    return dataExtend
"""

'\ndef DatosSinteticosNew(data,data_0,data_1, data_2, data_3, data_4, data_5):\n    n_filas=10000\n    nuevos_datos=np.empty([n_filas,len(data.columns)])\n    ColmunaY = np.random.randint(0, 6, n_filas)\n    print(ColmunaY)\n    for i in range(len(data.columns)-1):\n        for j in range(0,n_filas):\n            if ColmunaY[j]==0:\n                columna_i =data_0.iloc[:,i]\n\n            if ColmunaY[j]==1:\n                columna_i =data_1.iloc[:,i]\n                \n            if ColmunaY[j]==2:\n                columna_i =data_2.iloc[:,i]\n                \n            if ColmunaY[j]==3:\n                columna_i =data_3.iloc[:,i]\n               \n            if ColmunaY[j]==4:\n                columna_i =data_4.iloc[:,i]\n\n            if ColmunaY[j]==5:\n                columna_i =data_5.iloc[:,i]\n\n            muestras_i=np.random.choice(columna_i,1,replace=True)\n            if(ColmunaY[j] == 0):\n                muestras_i=muestras_i.round(5)\n            else:\n       

In [ ]:
#generamos una lista de dataframes a partir del dataset original de acuerdo a la clasificacion 
ListaData=[]
for i in range(0,len(value)):
    ListaData.append(data[data['Type'] == i])

In [ ]:
#ingresamos el dataset orginal,lista de dataFrames, cantidad de datos sinteticos y numero de clases
data=DatosSinteticosNew(data,ListaData,10000,6)
data.to_csv('Optimizado.csv',index=False)

[0 2 1 ... 4 1 0]
[[ 1.52227 12.78     3.57    ...  0.       0.       0.     ]
 [ 1.518   12.16     3.4     ...  0.       0.17     2.     ]
 [ 1.516   13.1      3.66    ...  0.       0.21     1.     ]
 ...
 [ 1.519   14.09     0.      ...  0.       0.       4.     ]
 [ 1.517   13.09     3.56    ...  0.       0.       1.     ]
 [ 1.51905 13.05     3.65    ...  0.       0.       0.     ]]


In [ ]:
"""from sklearn.datasets import make_regression
from sklearn.linear_model import  LinearRegression

#Dataset original
num_filas=data.shape[1]
#ajuste al medelo deregresion lineal con datos originales
model = LinearRegression()
model.fit(data.iloc[:,:num_filas-1],data.iloc[:,num_filas-1])
#Generar datos sinteticos
num_ejemplos =10000
X,y=make_regression(n_samples=num_ejemplos,n_features=num_filas-1,noise=0.1)
#ajuste de datos sinteticos utilizando el metodo de regresion lineal
X=np.round(X)
y=model.predict(X)
y=np.round(y)
#combiar la caracteristicas y la variablidad objetioc en un solo DataFrame
data_sintetico=pd.DataFrame(np.column_stack((X, y)),columns=data.columns)
#Guardadar los datos sinteticos en un archivo Csv
data_sintetico.to_csv('datos_sintetico.csv',index=False)
"""

"from sklearn.datasets import make_regression\nfrom sklearn.linear_model import  LinearRegression\n\n#Dataset original\nnum_filas=data.shape[1]\n#ajuste al medelo deregresion lineal con datos originales\nmodel = LinearRegression()\nmodel.fit(data.iloc[:,:num_filas-1],data.iloc[:,num_filas-1])\n#Generar datos sinteticos\nnum_ejemplos =10000\nX,y=make_regression(n_samples=num_ejemplos,n_features=num_filas-1,noise=0.1)\n#ajuste de datos sinteticos utilizando el metodo de regresion lineal\nX=np.round(X)\ny=model.predict(X)\ny=np.round(y)\n#combiar la caracteristicas y la variablidad objetioc en un solo DataFrame\ndata_sintetico=pd.DataFrame(np.column_stack((X, y)),columns=data.columns)\n#Guardadar los datos sinteticos en un archivo Csv\ndata_sintetico.to_csv('datos_sintetico.csv',index=False)\n"

In [ ]:
"""
from sklearn.mixture import GaussianMixture

def DatosSinteticos(data):
    n_filas = 10000
    nuevos_datos = np.empty([n_filas, len(data.columns)])
    for i in range(len(data.columns)):
        columna_i = data.iloc[:, i]
        media_i = np.mean(columna_i)
        desv_i = np.std(columna_i)
        #muestras_i = np.random.normal(media_i, desv_i, n_filas)
        muestras_i = np.random.lognormal(mean=np.log(media_i), sigma=desv_i, size=n_filas)
        if i==0:
            nuevos_datos[:, i] = np.abs(muestras_i).round(4)
        if i==9:
            muestras_i=np.abs(muestras_i)
            muestras_i=np.round(muestras_i).astype(int)
            nuevos_datos[:, i] = muestras_i
        if i!=0 and i!=9:
            nuevos_datos[:, i] = np.abs(muestras_i).round(2)
            
    columnas = list(data.columns)
    nuevos_datos_df = pd.DataFrame(nuevos_datos, columns=columnas)
    dataExtend = pd.concat([data, nuevos_datos_df], axis=0)
    
    # Validación de datos
    gmm = GaussianMixture(n_components=len(data.columns))
    gmm.fit(dataExtend)
    scores = gmm.score_samples(dataExtend)
    threshold = np.percentile(scores, 5)
    dataExtend = dataExtend[scores >= threshold]
    
    return dataExtend

data=DatosSinteticos(data)
data.to_csv('data_sintetico.csv',index=False)
"""

"\nfrom sklearn.mixture import GaussianMixture\n\ndef DatosSinteticos(data):\n    n_filas = 10000\n    nuevos_datos = np.empty([n_filas, len(data.columns)])\n    for i in range(len(data.columns)):\n        columna_i = data.iloc[:, i]\n        media_i = np.mean(columna_i)\n        desv_i = np.std(columna_i)\n        #muestras_i = np.random.normal(media_i, desv_i, n_filas)\n        muestras_i = np.random.lognormal(mean=np.log(media_i), sigma=desv_i, size=n_filas)\n        if i==0:\n            nuevos_datos[:, i] = np.abs(muestras_i).round(4)\n        if i==9:\n            muestras_i=np.abs(muestras_i)\n            muestras_i=np.round(muestras_i).astype(int)\n            nuevos_datos[:, i] = muestras_i\n        if i!=0 and i!=9:\n            nuevos_datos[:, i] = np.abs(muestras_i).round(2)\n            \n    columnas = list(data.columns)\n    nuevos_datos_df = pd.DataFrame(nuevos_datos, columns=columnas)\n    dataExtend = pd.concat([data, nuevos_datos_df], axis=0)\n    \n    # Validación 

In [ ]:
value = list(data["Segmentation"].unique())
print("numero de clases",value)
data.info()

numero de clases [0.0, 1.0, 2.0, 3.0]
<class 'pandas.core.frame.DataFrame'>
Index: 16665 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           16665 non-null  float64
 1   Ever_Married     16665 non-null  float64
 2   Age              16665 non-null  float64
 3   Graduated        16665 non-null  float64
 4   Profession       16665 non-null  float64
 5   Work_Experience  16665 non-null  float64
 6   Spending_Score   16665 non-null  float64
 7   Family_Size      16665 non-null  float64
 8   Var_1            16665 non-null  float64
 9   Segmentation     16665 non-null  float64
dtypes: float64(10)
memory usage: 1.4 MB


In [ ]:

from sklearn.mixture import GaussianMixture
def DatosSinteticos(data):
    n_filas = 10000
    nuevos_datos = np.empty([n_filas, len(data.columns)])
    for i in range(len(data.columns)):
        columna_i = data.iloc[:, i]
        media_i = np.mean(columna_i)
        desv_i = np.std(columna_i)
        muestras_i = np.random.lognormal(mean=np.log(media_i), sigma=desv_i, size=n_filas)
        muestras_i = np.abs(muestras_i) # Convertir cualquier valor negativo en su valor absoluto
        nuevos_datos[:, i] = muestras_i
    columnas = list(data.columns)
    nuevos_datos_df = pd.DataFrame(nuevos_datos, columns=columnas)
    dataExtend = pd.concat([data, nuevos_datos_df], axis=0)
    
    # Validación de datos
    gmm = GaussianMixture(n_components=len(data.columns))
    gmm.fit(dataExtend)
    scores = gmm.score_samples(dataExtend)
    threshold = np.percentile(scores, 5)
    dataExtend = dataExtend[scores >= threshold]
    
    return dataExtend


In [ ]:
def DatosSinteticos(data):
    n_filas=10000
    nuevos_datos=np.empty([n_filas,len(data.columns)])
    for i in range(len(data.columns)):
        columna_i =data.iloc[:,i]
        muestras_i=np.random.choice(columna_i,n_filas,replace=True)
        nuevos_datos[:,i]=muestras_i.round()
    columnas =list(data.columns)
    nuevos_datos_df=pd.DataFrame(nuevos_datos,columns=columnas)
    dataExtend =pd.concat([data,nuevos_datos_df],axis=0)
    #print(dataExtend.tail())
    return dataExtend

In [ ]:
data=data.to_numpy()

In [ ]:
num_labels = 6
X, y = data[0:10000, :9], data[0:10000, 9]
Xprueba=data[10000:,:9]
yprueba=data[10000:,9]
m = y.size
print(X.shape)

(10000, 9)


In [ ]:
print(Xprueba.shape)
print(X.shape)

(6665, 9)
(10000, 9)


In [ ]:
print(X)
print(y)

[[ 0.    0.   22.   ...  0.    4.    0.  ]
 [ 1.    1.   67.   ...  0.    1.    1.  ]
 [ 0.    1.   67.   ...  1.    2.    1.  ]
 ...
 [ 0.    1.   77.   ...  0.    5.    3.  ]
 [ 1.    0.   52.   ...  0.    4.19  1.  ]
 [ 1.    1.   49.   ...  1.    1.13  1.  ]]
[0. 1. 1. ... 1. 3. 2.]


In [ ]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate([np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0], axis=1)

# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 3

In [ ]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
def lrCostFunction(theta, X, y, lambda_):
 
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size
    epsilon = 1e-8 
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h+epsilon)) - (1 - y).dot(np.log(1 - h+epsilon))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

#### 1.3.3 Vectorización regularizada de la regresión logística




In [ ]:
J, grad = lrCostFunction(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 2.534819
-----------------------
Gradientes:
 [0.146561, -0.548558, 0.724722, 1.398003]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


<a id="section2"></a>
### 1.4 Clasificacion One-vs-all


In [ ]:
def oneVsAll(X, y, num_labels, lambda_):

    # algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 800}
        res = optimize.minimize(lrCostFunction, 
                                initial_theta, 
                                (X, (y == c), lambda_), 
                                jac=True, 
                                method='CG',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [ ]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [ ]:
lambda_ = 0.03 #mejor lambda es 0.002
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)
print(all_theta)

(6, 10)
[[ 2.06380499e+00 -3.92452335e-01 -7.57306442e-01 -3.73102166e-02
  -1.26472704e+00 -1.20273129e-01  3.26337254e-02 -7.42880306e-01
   1.09849099e-01 -2.84265529e-02]
 [-2.31283993e+00  1.25833082e-01  4.15097624e-01  1.10073114e-02
   3.56377862e-01  4.91171370e-02 -1.29347827e-02  1.71378403e-01
  -8.45763592e-02  4.89961076e-02]
 [-3.80891801e+00  1.88944086e-01  4.50259133e-01  1.51574904e-02
   1.15167030e+00 -9.29618821e-03 -3.51852266e-02  6.59696396e-01
   1.41730048e-01 -2.17014097e-03]
 [-8.43975847e-01  1.76875468e-02  1.32217358e-01  3.33763355e-03
  -5.49268467e-02  1.01994130e-01  2.03890758e-02 -4.41709530e-01
  -2.17903770e-01 -3.56449597e-02]
 [-2.30627463e-02 -1.05659301e-02 -1.37705813e-02 -1.00198634e+00
  -1.47808910e-02 -7.62261126e-02 -6.05068215e-02 -1.50206513e-02
  -6.57612762e-02 -3.24050958e-02]
 [-2.30627463e-02 -1.05659301e-02 -1.37705813e-02 -1.00198634e+00
  -1.47808910e-02 -7.62261126e-02 -6.05068215e-02 -1.50206513e-02
  -6.57612762e-02 -3.2405

In [ ]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = Xprueba[40:41, :].copy()
y_p=yprueba[40]
print("se espera de clase :",y_p)
XPrueba = np.concatenate([np.ones((1, 1)), XPrueba], axis=1)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print("La clase es: ", p)

#displayData(X[1002:1003, :])

(10000, 9)
Precision del conjuto de entrenamiento: 47.73%
se espera de clase : 0.0
La clase es:  [2]
